## Analyzing the impact of weather on bike trip duration using OLS regression method



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.4f}'.format

pd.set_option('display.max_columns', 999)

# All Libraries needed for Regression imported

# All Dataframes as CSV needed for regression

bike_journey_data=pd.read_csv('bike_journey_data_27_mar_2020.csv')
weather_data=pd.read_csv('weather_data_27_mar_2020.csv')

In [2]:
'''weather_data=weather_data.drop(columns=['Date','Date Converted','Hours','id'])
weather_data=weather_data.set_index('id_Hours')

from scipy import stats
import numpy as np

df = pd.DataFrame(weather_data)
df.head()

z = np.abs(stats.zscore(df))
print(z)
threshold = 4
print(np.where(z > 4))

df = df[(z < 4).all(axis=1)]
print(df.shape)'''

"weather_data=weather_data.drop(columns=['Date','Date Converted','Hours','id'])\nweather_data=weather_data.set_index('id_Hours')\n\nfrom scipy import stats\nimport numpy as np\n\ndf = pd.DataFrame(weather_data)\ndf.head()\n\nz = np.abs(stats.zscore(df))\nprint(z)\nthreshold = 4\nprint(np.where(z > 4))\n\ndf = df[(z < 4).all(axis=1)]\nprint(df.shape)"

In [3]:
'''x=df['precipIntensity']
plt.boxplot(x, notch=None)'''

"x=df['precipIntensity']\nplt.boxplot(x, notch=None)"

In [4]:
'''statistics = df['precipIntensity'].describe()
statistics'''

"statistics = df['precipIntensity'].describe()\nstatistics"

In [5]:
'''# Collpase the bike and weather dataset by trips by hour and sum the number of trips and total duration of all trips by hour
reg_merge= bike_journey_data.merge(right=weather_data,
                             left_on = 'id_hours',
                             right_on = 'id_Hours')

reg_merge['bike_trips_by_hour'] = 1
def bike_trips_by_hour(reg_merge, renamed_count):
    bike_trips_by_hour = reg_merge.loc[:, ('id_hours', 'bike_trips_by_hour')].groupby(['id_hours']).sum()
    bike_trips_by_hour.reset_index(inplace=True)
    bike_trips_by_hour.rename(columns={'bike_trips_by_hour': str(renamed_count)}, inplace=True)
    return bike_trips_by_hour
bike_trips_by_hour = bike_trips_by_hour(reg_merge, 'bike_trips_by_hour')

reg_merge3= reg_merge.merge(right=bike_trips_by_hour,
                             left_on = 'id_hours',
                             right_on = 'id_hours')

reg_merge3['Duration in hours'] = 1
def duration_by_hour(reg_merge3, renamed_count):
    duration_by_hour = reg_merge3.loc[:, ('id_hours', 'Duration in minutes')].groupby(['id_hours']).sum()
    duration_by_hour.reset_index(inplace=True)
    duration_by_hour.rename(columns={'Duration in minutes': str(renamed_count)}, inplace=True)
    return duration_by_hour
duration_by_hour = duration_by_hour(reg_merge3, 'duration_by_hour')

regression_dataset= reg_merge3.merge(right=duration_by_hour,
                             left_on = 'id_hours',
                             right_on = 'id_hours')

regression_dataset['Avg trip duration']=regression_dataset['duration_by_hour']/regression_dataset['bike_trips_by_hour_y']'''


In [8]:
'''df_month=pd.get_dummies(regression_dataset.Month_x)
df_month.columns=['Jan','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
#df_month=df_month.astype(int)
regression_dataset = regression_dataset.join(df_month)'''

In [9]:
'''##Generating dummy variables based on temperature

regression_dataset["temp_0"] = regression_dataset['temperature'].apply(lambda x: 1 if x <=0 else 0)
regression_dataset["temp_5"] = regression_dataset['temperature'].apply(lambda x: 1 if x>0 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=5 else 0)
regression_dataset["temp_10"] = regression_dataset['temperature'].apply(lambda x: 1 if x>5 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=10 else 0)
regression_dataset["temp_15"] = regression_dataset['temperature'].apply(lambda x: 1 if x>10 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=15 else 0)

regression_dataset["temp_20"] = regression_dataset['temperature'].apply(lambda x: 1 if x>15 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=20 else 0)
regression_dataset["temp_25"] = regression_dataset['temperature'].apply(lambda x: 1 if x>20 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=25 else 0)

regression_dataset["temp_30"] = regression_dataset['temperature'].apply(lambda x: 1 if x>25 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=30 else 0)
regression_dataset["temp_35"] = regression_dataset['temperature'].apply(lambda x: 1 if x>30 else 0) & regression_dataset['temperature'].apply(lambda x: 1 if x<=35 else 0)
regression_dataset["temp_40"] = regression_dataset['temperature'].apply(lambda x: 1 if x>35 else 0)
regression_dataset['Jan']=regression_dataset['Jan'].replace(1,0)'''


In [10]:
'''##Generating a dummy variables for humidity and rain based on the mean value (0.6497)
regression_dataset["relative_humidity"] = regression_dataset['humidity'].apply(lambda x: 1 if x >=0.6497 else 0)
regression_dataset["rain"] = regression_dataset['precipIntensity'].apply(lambda x: 1 if x >0.0697 else 0)'''

In [9]:
#x=regression_dataset['humidity']
#plt.boxplot(x, notch=None)

In [10]:
'''import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

y = 'Avg trip duration'
x_1 = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35']
x_2 = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35','relative_humidity', 'rain','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
x_3 = ['temp_0','temp_5', 'temp_10', 'temp_15', 'temp_20', 'temp_25', 'temp_30', 'temp_35','humidity','precipIntensity','Feb','Mar', 'Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']

#model_Simple = sm.OLS(temperature_10_dataset[y], temperature_10_dataset[x]).fit()

model_1 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x_1])).fit()
model_1.summary()
model_2 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x_2])).fit()
model_3 = sm.OLS(regression_dataset[y], sm.add_constant(regression_dataset[x_3])).fit()'''

/Users/albegjonbalaj/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [6]:
'''## descriptive statistics
statistics_1 = regression_dataset['bike_trips_by_hour_y'].describe()
statistics_1

statistics_2 = regression_dataset['Avg trip duration'].describe()
statistics_2

statistics_3 = regression_dataset['temperature'].describe()
statistics_3

statistics_4 = regression_dataset['humidity'].describe()
statistics_4

statistics_5 = regression_dataset['windSpeed'].describe()
statistics_5'''



/Users/albegjonbalaj/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [11]:
'''dfoutput = summary_col([model_1, model_2, model_3],stars=True)
print(dfoutput)'''


                  Avg trip duration I Avg trip duration II Avg trip duration III
--------------------------------------------------------------------------------
Apr                                   1.2335***            0.9435***            
                                      (0.0116)             (0.0115)             
Aug                                   -1.9714***           -1.3144***           
                                      (0.0129)             (0.0128)             
Dec                                   2.5049***            2.7019***            
                                      (0.0113)             (0.0113)             
Feb                                   0.6096***            0.8480***            
                                      (0.0112)             (0.0111)             
Jul                                   -3.4095***           -2.5703***           
                                      (0.0129)             (0.0129)             
Jun                        

In [235]:
weather_data.head()

,Date,temperature,precipIntensity,humidity,windSpeed,visibility,Date.1,Date Converted,Hours,Day,Week Day,Month,Year,id,id_Hours
0,2019-01-02 00:00:00,5.1100,0.0000,0.6900,7.6200,16.0930,2019-01-02 00:00:00,2019-01-02,0,2,2,1,2019,2019-1-2,2019-01-02-0
1,2019-01-02 01:00:00,4.4600,0.0000,0.7100,7.5200,16.0930,2019-01-02 01:00:00,2019-01-02,1,2,2,1,2019,2019-1-2,2019-01-02-1
2,2019-01-02 02:00:00,3.7700,0.0000,0.7300,7.6000,16.0930,2019-01-02 02:00:00,2019-01-02,2,2,2,1,2019,2019-1-2,2019-01-02-2
3,2019-01-02 03:00:00,3.1700,0.0000,0.7500,7.6600,16.0930,2019-01-02 03:00:00,2019-01-02,3,2,2,1,2019,2019-1-2,2019-01-02-3
4,2019-01-02 04:00:00,2.7200,0.0000,0.7600,7.6200,16.0930,2019-01-02 04:00:00,2019-01-02,4,2,2,1,2019,2019-1-2,2019-01-02-4


In [50]:
bike_journeys=bike_journey_data.loc[bike_journey_data['Duration in minutes'] < 200]
bike_journeys['c'] = 1
daily_bike_journeys = bike_journeys.loc[:, ('c', 'id', 'id_hours', 'Duration in minutes')].groupby(['id_hours']).sum()
daily_bike_journeys.reset_index(inplace=True)
daily_bike_journeys.rename(columns={'c': 'Bike trips', 'id': 'id', 'Hours': 'Hours'}, inplace=True)
#print (daily_bike_journeys.shape)

reg_data= daily_bike_journeys.merge(right=weather_data,
                             left_on = 'id_hours',
                             right_on = 'id_Hours')

reg_data['Avg trip duration']=reg_data['Duration in minutes']/reg_data['Bike trips']

df_month=pd.get_dummies(reg_data.Month)
df_month.columns=['January 2019','February 2019','March 2019', 'April 2019','May 2019','June 2019','July 2019',
                  'August 2019','September 2019','October 2019','November 2019','December 2019']
reg_data = reg_data.join(df_month)

reg_data["Temp - 0(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x <=0 else 0)
reg_data["Temp - 5(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>0 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=5 else 0)
reg_data["Temp - 10(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>5 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=10 else 0)
reg_data["Temp - 15(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>10 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=15 else 0)
reg_data["Temp - 20(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>15 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=20 else 0)
reg_data["Temp - 25(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>20 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=25 else 0)
reg_data["Temp - 30(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>25 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=30 else 0)#reg_data["temp_35"] = reg_data['temperature'].apply(lambda x: 1 if x>30 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=35 else 0)
reg_data["Temp - 35(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>30 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=35 else 0)
#reg_data["temp_40"] = reg_data['temperature'].apply(lambda x: 1 if x>35 else 0)
#reg_data['Jan']=reg_data['Jan'].replace(1,0)

reg_data["Relative Humidity"] = reg_data['humidity'].apply(lambda x: 1 if x >=0.6497 else 0)
reg_data["Rain"] = reg_data['precipIntensity'].apply(lambda x: 1 if x >0.0697 else 0)
reg_data["Wind Speed (MPH)"] = reg_data['windSpeed'].apply(lambda x: 1 if x >8.1400 else 0)

reg_data["Weekend"] = reg_data['Week Day'].apply(lambda x: 1 if x >4 else 0)

reg_data["Peak travel hours"] = reg_data['Hours'].apply(lambda x: 1 if x >=7 and x<10 else (1 if x >= 16 and x < 20 else 0))

reg_data=reg_data.set_index('id')

holidays = ['2019-1-1', '2019-4-19', '2019-4-22', '2019-5-6', '2019-5-27', '2019-8-26', '2019-12-25', '2019-12-26']
reg_data['Holidays'] = reg_data.index.get_level_values(0).isin(holidays).astype(int)


/Users/albegjonbalaj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

y1 = 'Avg trip duration'
y2 = 'Bike trips'

x_0=['temperature']

x_1 = ["Temp - 15(°C)","Relative Humidity", "Rain", 'Wind Speed (MPH)','February 2019','March 2019', 
       'April 2019','May 2019','June 2019','July 2019','August 2019','September 2019','October 2019',
       'November 2019','December 2019', "Peak travel hours",
       "Holidays", "Weekend"]

x_2 = ["Temp - 0(°C)", "Temp - 5(°C)", "Temp - 10(°C)", "Temp - 15(°C)", "Temp - 20(°C)", "Temp - 25(°C)",
       "Temp - 30(°C)", "Temp - 35(°C)"]

x_3 = ["Temp - 0(°C)", "Temp - 5(°C)", "Temp - 10(°C)", "Temp - 15(°C)", "Temp - 20(°C)", "Temp - 25(°C)",
       "Temp - 30(°C)", "Temp - 35(°C)", "Relative Humidity", "Rain", 'Wind Speed (MPH)','February 2019','March 2019', 
       'April 2019','May 2019','June 2019','July 2019','August 2019','September 2019','October 2019',
       'November 2019','December 2019', "Peak travel hours",
       "Holidays", "Weekend"]

#model_0 = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x_0])).fit()
model_1 = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x_1])).fit()
model_2 = sm.OLS(reg_data[y2], sm.add_constant(reg_data[x_3])).fit()
model_3 = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x_3])).fit()

In [56]:
dfoutput = summary_col([model_3, model_2],stars=True, 
                       regressor_order=['temperature',"Temp - 0(°C)", "Temp - 5(°C)", "Temp - 10(°C)", "Temp - 15(°C)", "Temp - 20(°C)", "Temp - 25(°C)",
                                        "Temp - 30(°C)", "Temp - 35(°C)", "Relative Humidity", "Rain", 'Wind Speed (MPH)', "Weekend",'Holidays',
                                        "Peak travel hours",'February 2019','March 2019', 
                                        'April 2019','May 2019','June 2019','July 2019','August 2019','September 2019','October 2019',
                                        'November 2019','December 2019'])

dfoutput.add_title('Table 1: OLS Model for Weather Impacts on Hourly Average Trip Duration and Bike Trips')

print(dfoutput)

Table 1: OLS Model for Weather Impacts on Hourly Average Trip Duration and Bike Trips
                  Avg trip duration   Bike trips 
-------------------------------------------------
Temp - 0(°C)      -5.8459***        -1350.6881***
                  (1.2715)          (317.4773)   
Temp - 5(°C)      -5.3745***        -1204.2040***
                  (1.2220)          (305.1360)   
Temp - 10(°C)     -4.7534***        -1071.7389***
                  (1.2161)          (303.6538)   
Temp - 15(°C)     -3.5539***        -783.7574*** 
                  (1.2118)          (302.5910)   
Temp - 20(°C)     -2.0839*          -389.8567    
                  (1.2075)          (301.5165)   
Temp - 25(°C)     0.1949            82.4313      
                  (1.2072)          (301.4314)   
Temp - 30(°C)     0.4426            300.1087     
                  (1.2351)          (308.3908)   
Temp - 35(°C)     1.3261            430.6938     
                  (1.2767)          (318.7901)   
Relative Humid